In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
import os

In [2]:
os.listdir()

['.DS_Store',
 '.ipynb_checkpoints',
 '2way_anova.ipynb',
 '2wayAnova.csv',
 'bin',
 'metadata.csv',
 'metadata2_Experimental_design.csv',
 'padj_2way_anova.R',
 'padj_on_all_2wayAnova.csv',
 'padj_on_annot_2wayAnova.csv']

In [14]:
data_df = pd.read_csv("../../X0_log2_transformed_log2FC/featab_simp_norm_annot_rm_TC-5_log2.csv",index_col=0)
t_data_df = data_df.T

In [15]:
t_data_df.index

Index(['C2_B6.1_Tfh_007', 'C2_B6.2_Tfh_008', 'C2_B6.4_Tfh_009',
       'C2_B6.6_Tfh_010', 'C2_B6.7_Tfh_011', 'C2_B6.8_Tfh_012',
       'C2_B6.1_Tn_001', 'C2_B6.2_Tn_002', 'C2_B6.4_Tn_003', 'C2_B6.6_Tn_004',
       'C2_B6.7_Tn_005', 'C2_B6.8_Tn_006', 'C1_B6.1_Tfh_004', 'C1_B6.1_Tn_001',
       'C1_B6.2_Tfh_005', 'C1_B6.2_Tn_002', 'C1_B6.3_Tfh_006',
       'C1_B6.3_Tn_003', 'C1_TC.1_Tfh_010', 'C1_TC.1_Tn_007',
       'C1_TC.2_Tfh_011', 'C1_TC.2_Tn_008', 'C1_TC.2_Tn_009',
       'C1_TC.3_Tfh_012', 'C2_TC.1_Tfh_018', 'C2_TC.2_Tfh_019',
       'C2_TC.3_Tfh_020', 'C2_TC.4_Tfh_021', 'C2_TC.1_Tn_013',
       'C2_TC.2_Tn_014', 'C2_TC.3_Tn_015', 'C2_TC.4_Tn_016'],
      dtype='object')

In [16]:
t_data_df.index

Index(['C2_B6.1_Tfh_007', 'C2_B6.2_Tfh_008', 'C2_B6.4_Tfh_009',
       'C2_B6.6_Tfh_010', 'C2_B6.7_Tfh_011', 'C2_B6.8_Tfh_012',
       'C2_B6.1_Tn_001', 'C2_B6.2_Tn_002', 'C2_B6.4_Tn_003', 'C2_B6.6_Tn_004',
       'C2_B6.7_Tn_005', 'C2_B6.8_Tn_006', 'C1_B6.1_Tfh_004', 'C1_B6.1_Tn_001',
       'C1_B6.2_Tfh_005', 'C1_B6.2_Tn_002', 'C1_B6.3_Tfh_006',
       'C1_B6.3_Tn_003', 'C1_TC.1_Tfh_010', 'C1_TC.1_Tn_007',
       'C1_TC.2_Tfh_011', 'C1_TC.2_Tn_008', 'C1_TC.2_Tn_009',
       'C1_TC.3_Tfh_012', 'C2_TC.1_Tfh_018', 'C2_TC.2_Tfh_019',
       'C2_TC.3_Tfh_020', 'C2_TC.4_Tfh_021', 'C2_TC.1_Tn_013',
       'C2_TC.2_Tn_014', 'C2_TC.3_Tn_015', 'C2_TC.4_Tn_016'],
      dtype='object')

In [40]:
meta_df = pd.read_csv("metadata_SampleInfo_mod_rmTC-5.csv") # you may need to change index_col to give the right index

In [42]:
meta_df.index = [x.replace("-",".") for x in meta_df['SampleID']]

In [29]:
t_data_df.shape

(32, 111)

In [43]:
pval_l = []
for i in range(t_data_df.shape[1]):
    m_df = pd.merge(meta_df.loc[:,['Genotype','Celltype']],pd.DataFrame(t_data_df.iloc[:,i]) , left_index=True, right_index=True)
    m_df.columns = ['Genotype','Celltype','log2Int']

    mod = smf.ols(formula='log2Int ~ C(Genotype) * C(Celltype)', data=m_df)

    #https://www.statsmodels.org/dev/example_formulas.html
    res = mod.fit()
    table = sm.stats.anova_lm(res, typ=2) # Type 2 Anova DataFrame

    pval_l.append(table.iloc[2,3])

In [44]:
pval_df = pd.DataFrame(pval_l,columns = ['2wayAnova_pval'],index = data_df.index)

In [48]:
pval_df.to_csv("2wayAnova.csv")

In [49]:
merged_df = pd.concat([data_df,pval_df],axis = 1) 

In [50]:
merged_df.to_csv("2wayAnova_merged_wt_data_table.csv")